# 1 Exercise 2

Description

In [16]:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt

from typing import Tuple

In [22]:
def optimal_percentage(price: float, cost: float, holding_cost: float) -> float:
    return (price - cost)/(price + holding_cost)

In [9]:
def root_mean_squared_error(quantity_estimation_vector: np.ndarray, known_optimal_quantity: float) -> float:
    estimation_error: np.ndarray = quantity_estimation_vector - known_optimal_quantity
    squared_error: float = np.square(estimation_error).sum()

    vector_size: int = quantity_estimation_vector.size
    mean_squared_error: float = squared_error/vector_size
    return np.sqrt(mean_squared_error)


In [14]:
def profit_loss_ratio(profit_estimation_vecor: np.ndarray, known_maximum_profit: float) -> float:
    ratio: np.ndarray = (known_maximum_profit - profit_estimation_vecor)/known_maximum_profit
    vector_size = profit_estimation_vecor.size
    return np.absolute(ratio).sum()/vector_size


In [21]:
def normal_parametric_estimation(data_vector: np.ndarray) -> Tuple[float, float]:
    mean: float = data_vector.mean()
    var: float = data_vector.var()
    return (mean, var)

def parametric_normal_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    mean, var = normal_parametric_estimation(data_vector)
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    inv_cdf_quantity: float =  sts.norm.ppf(optimal_percent, loc=mean, scale=var)

    return inv_cdf_quantity

In [23]:
def log_normal_parametric_estimation(data_vector: np.ndarray) -> Tuple[float, float]:
    # Dit moet nog even gecheckt worden, ik heb geen idee of dit is hoe je het berekent. Micha
    mean: float = np.log(data_vector).mean()
    var: float = np.log(data_vector).var()
    return (mean, var)

def parametric_log_normal_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    mean, var = normal_parametric_estimation(data_vector)
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    inv_cdf_quantity: float = sts.lognorm.ppf(optimal_percent, loc=np.exp(mean), scale=np.exp(var))

    return inv_cdf_quantity

In [24]:
def non_parametric_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    sorted_data_vector: np.ndarray = data_vector.sort()
    vector_size = data_vector.size
    optimal_percent = optimal_percentage(price, cost, holding_cost)
    optimal_arg_value = np.ceil(optimal_percentage * vector_size)

    return data_vector[optimal_arg_value]

In [31]:
def normal_monte_carlo_simulation(number, target_surface, location, scale):
    
    pass

In [32]:
def normal():
    location = 100
    scale = 10
    number = [10, 50, 100, 200]
    target_surface = [0.01, 0.05, 0.1, 0.3, 0.7, 0.9, 0.95, 0.99]

    for n in number:
        for t in target_surface:
            normal_monte_carlo_simulation(n, t, location, scale)

In [30]:
normal()